# Step 1 : data cleaning

No duplicates, no NANs, no text data, no features that have too strong correlation between them.

In [245]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

df = pd.read_csv("../final_dataset.csv")

In [246]:
df

,Url,BathroomCount,BedroomCount,ConstructionYear,Country,District,Fireplace,FloodingZone,Furnished,Garden,...,RoomCount,ShowerCount,StateOfBuilding,SubtypeOfProperty,SurfaceOfPlot,SwimmingPool,Terrace,ToiletCount,TypeOfProperty,TypeOfSale
0,https://www.immoweb.be/en/classified/apartment...,1.0,1.0,1969.0,Belgium,Brugge,NaN,NaN,NaN,NaN,...,1.0,0.0,GOOD,flat_studio,NaN,NaN,1.0,1.0,2,residential_sale
1,https://www.immoweb.be/en/classified/house/for...,6.0,13.0,1920.0,Belgium,Tournai,NaN,NaN,0.0,NaN,...,31.0,NaN,GOOD,apartment_block,130.0,NaN,NaN,5.0,1,residential_sale
2,https://www.immoweb.be/en/classified/house/for...,2.0,4.0,2008.0,Belgium,Brugge,NaN,NON_FLOOD_ZONE,1.0,NaN,...,NaN,0.0,GOOD,house,0.0,NaN,NaN,2.0,1,residential_sale
3,https://www.immoweb.be/en/classified/house/for...,1.0,4.0,NaN,Belgium,Veurne,NaN,NaN,NaN,1.0,...,NaN,NaN,TO_BE_DONE_UP,house,170.0,0.0,1.0,2.0,1,residential_sale
4,https://www.immoweb.be/en/classified/apartment...,0.0,2.0,1972.0,Belgium,Hasselt,NaN,NON_FLOOD_ZONE,NaN,NaN,...,1.0,0.0,AS_NEW,apartment,NaN,NaN,1.0,1.0,2,residential_sale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118709,https://www.immoweb.be/en/classified/house/for...,1.0,3.0,2024.0,Belgium,Gent,NaN,NaN,NaN,1.0,...,4.0,NaN,GOOD,house,234.0,NaN,NaN,0.0,1,residential_sale
118710,https://www.immoweb.be/en/classified/house/for...,4.0,4.0,NaN,Belgium,Antwerp,NaN,NaN,0.0,NaN,...,NaN,NaN,JUST_RENOVATED,apartment_block,202.0,NaN,NaN,NaN,1,residential_sale
118711,https://www.immoweb.be/en/classified/apartment...,1.0,2.0,NaN,Belgium,Antwerp,NaN,NON_FLOOD_ZONE,NaN,NaN,...,NaN,NaN,GOOD,apartment,NaN,0.0,1.0,1.0,2,residential_sale
118712,https://www.immoweb.be/en/classified/apartment...,1.0,2.0,NaN,Belgium,Brugge,NaN,POSSIBLE_FLOOD_ZONE,NaN,NaN,...,8.0,NaN,NaN,apartment,NaN,NaN,1.0,NaN,2,residential_sale


## No duplicates

All 118714 rows have distinct urls, so I don't think there are any duplicates in rows.

## No text data

We'll use mapping to turn categorical data into numerical data. (First, we arrange the categories in ascending order.)

In [247]:
subtype_mapping = {
    'flat_studio': 1,
    'kot': 2,
    'apartment': 3,
    'service_flat': 4,
    'ground_floor': 5,
    'loft': 6,
    'duplex': 7,
    'house': 8,
    'bungalow': 9,
    'country_cottage': 10,
    'town_house': 11,
    'chalet': 12,
    'villa': 13,
    'penthouse': 14,
    'triplex': 15,
    'mixed_use_building': 16,
    'apartment_block': 17,
    'farmhouse': 18,
    'pavilion': 19,
    'show_house': 20,
    'mansion': 21,
    'exceptional_property': 22,
    'manor_house': 23,
    'castle': 24,
    'other_property': 25
}

type_of_sale_mapping = {
    'residential_monthly_rent' : 1,
    'annuity_monthly_amount' : 2,  
    'annuity_without_lump_sum' : 3,       
    'annuity_lump_sum' : 4,                 
    'homes_to_build' : 5,
    'residential_sale' : 6     
}

region_mapping = {
    'Wallonie' : 1,
    'Flanders' : 2,  
    'Brussels' : 3   
}

province_mapping = {
    'Hainaut' : 1,
    'Liège' : 2,
    'Luxembourg' : 3,
    'Namur' : 4,
    'Walloon Brabant' : 5,
    'Limburg' : 6,
    'East Flanders': 7,
    'Flemish Brabant': 8,
    'Antwerp': 9,
    'West Flanders' : 10,
    'Brussels' : 11
}

PEB_mapping = {
    'NO_DATA': 0,
    'G': 1, 
    'G_C': 1,
    'G_F': 1,  
    'F': 2,
    'F_C': 2,
    'F_D': 2,
    'F_E': 2,
    'E': 3,
    'E_C': 3,
    'E_D': 3,
    'D': 4,
    'C': 5,
    'B': 6,
    'B_A': 6,
    'A': 7,
    'A_A+': 7,
    'A+': 8,
    'A++': 9     
}

df['PEB'] = df['PEB'].fillna('NO_DATA')
df['PEB_num'] = df['PEB'].map(PEB_mapping)

df['SubtypeOfProperty_num'] = df['SubtypeOfProperty'].map(subtype_mapping)

df['TypeOfSale_num'] = df['TypeOfSale'].map(type_of_sale_mapping)

data = pd.DataFrame(df["LivingArea"])
living_area_imputer = IterativeImputer(random_state=0)
data_imputed = pd.DataFrame(living_area_imputer.fit_transform(data), columns=["LivingArea"])
df["LivingArea"] = data_imputed["LivingArea"].astype("int32")

df = df.dropna(subset=['Region','Province']) 
df['Region_num'] = df['Region'].map(region_mapping)   
df['Province_num'] = df['Province'].map(province_mapping)                            

C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\4217580158.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Region_num'] = df['Region'].map(region_mapping)
C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\4217580158.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Province_num'] = df['Province'].map(province_mapping)


In [248]:
df

,Url,BathroomCount,BedroomCount,ConstructionYear,Country,District,Fireplace,FloodingZone,Furnished,Garden,...,SwimmingPool,Terrace,ToiletCount,TypeOfProperty,TypeOfSale,PEB_num,SubtypeOfProperty_num,TypeOfSale_num,Region_num,Province_num
0,https://www.immoweb.be/en/classified/apartment...,1.0,1.0,1969.0,Belgium,Brugge,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,2,residential_sale,0,1,6,2,10
1,https://www.immoweb.be/en/classified/house/for...,6.0,13.0,1920.0,Belgium,Tournai,NaN,NaN,0.0,NaN,...,NaN,NaN,5.0,1,residential_sale,4,17,6,1,1
2,https://www.immoweb.be/en/classified/house/for...,2.0,4.0,2008.0,Belgium,Brugge,NaN,NON_FLOOD_ZONE,1.0,NaN,...,NaN,NaN,2.0,1,residential_sale,6,8,6,2,10
3,https://www.immoweb.be/en/classified/house/for...,1.0,4.0,NaN,Belgium,Veurne,NaN,NaN,NaN,1.0,...,0.0,1.0,2.0,1,residential_sale,2,8,6,2,10
4,https://www.immoweb.be/en/classified/apartment...,0.0,2.0,1972.0,Belgium,Hasselt,NaN,NON_FLOOD_ZONE,NaN,NaN,...,NaN,1.0,1.0,2,residential_sale,6,3,6,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118709,https://www.immoweb.be/en/classified/house/for...,1.0,3.0,2024.0,Belgium,Gent,NaN,NaN,NaN,1.0,...,NaN,NaN,0.0,1,residential_sale,0,8,6,2,7
118710,https://www.immoweb.be/en/classified/house/for...,4.0,4.0,NaN,Belgium,Antwerp,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,1,residential_sale,5,17,6,2,9
118711,https://www.immoweb.be/en/classified/apartment...,1.0,2.0,NaN,Belgium,Antwerp,NaN,NON_FLOOD_ZONE,NaN,NaN,...,0.0,1.0,1.0,2,residential_sale,4,3,6,2,9
118712,https://www.immoweb.be/en/classified/apartment...,1.0,2.0,NaN,Belgium,Brugge,NaN,POSSIBLE_FLOOD_ZONE,NaN,NaN,...,NaN,1.0,NaN,2,residential_sale,4,3,6,2,10


In [249]:
kitchen_mapping = {
    'NO_DATA' : 0,
    'NOT_INSTALLED': 1,
    'USA_UNINSTALLED': 2,
    'SEMI_EQUIPPED': 3,
    'USA_SEMI_EQUIPPED': 4,
    'INSTALLED': 5,
    'USA_INSTALLED': 6,
    'HYPER_EQUIPPED': 7,
    'USA_HYPER_EQUIPPED': 8
}

df['Kitchen'] = df['Kitchen'].fillna('NO_DATA')
df['Kitchen_num'] = df['Kitchen'].map(kitchen_mapping)

C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\3486637195.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Kitchen'] = df['Kitchen'].fillna('NO_DATA')
C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\3486637195.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Kitchen_num'] = df['Kitchen'].map(kitchen_mapping)


In [250]:
state_of_building_mapping = {
    'NO_DATA' : 0,
    'TO_RESTORE' : 1,
    'TO_RENOVATE' : 2,
    'TO_BE_DONE_UP' : 3,
    'GOOD' : 4,
    'AS_NEW' : 5,
    'JUST_RENOVATED' : 6
}
df['StateOfBuilding'] = df['StateOfBuilding'].fillna('NO_DATA')
df['StateOfBuilding_num'] = df['StateOfBuilding'].map(state_of_building_mapping)       

C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\2823316570.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['StateOfBuilding'] = df['StateOfBuilding'].fillna('NO_DATA')
C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\2823316570.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['StateOfBuilding_num'] = df['StateOfBuilding'].map(state_of_building_mapping)


## No NAN
Regarding missing values, depending on the nature of the feature, we'll either replace them with 0 or impute them.

In [251]:
fill_with_zeros = ['Furnished','Garden','Fireplace','SwimmingPool','Terrace','ToiletCount','ShowerCount','SurfaceOfPlot']
for i in fill_with_zeros:
    df[i] = df[i].fillna(0)

df.loc[df['Garden'] == 0, 'GardenArea'] = 0

C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\511567853.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i].fillna(0)


## Locality

On crée un dictionnaire avec les 5000 communes et la moyenne de leurs prix.

In [252]:
locality_dict = df.groupby('Locality')['Price'].mean().to_dict()
print(locality_dict)

{' Beez': 129000.0, ' Boncelles': 185000.0, ' Châtelineau': 120000.0, ' Couillet': 235000.0, ' Gilly': 89000.0, ' Koersel': 339000.0, ' Obourg': 215000.0, ' Suarlée': 349000.0, "'S GRAVENWEZEL": 585000.0, "'S-GRAVENBRAKEL": 248566.66666666666, "'S-GRAVENWEZEL": 650962.5, "'s Gravenwezel": 923263.6363636364, "'s-Gravenbrakel": 350000.0, "'s-Gravenwezel": 554900.0, '(Lutremange) Bastogne': 345000.0, '(Sainte-Marie) Libramont-Chevigny': 349000.0, '(Villance) Libin': 360000.0, '.': 550000.0, '1040 - Etterbeek': 1250.0, '1180 - Uccle': 2900.0, '3090 Overijse': 1300.0, 'AAIGEM': 270000.0, 'AALBEKE': 252000.0, 'AALST': 314814.8671875, 'AALTER': 213716.66666666666, 'AARSCHOT': 378141.42307692306, 'AARTRIJKE': 1050.0, 'AARTSELAAR': 411486.0869565217, 'AAT': 211500.0, 'ACHOUFFE': 299000.0, 'ADINKERKE': 219000.0, 'AFFLIGEM': 574900.0, 'AFSNEE': 971000.0, 'AGIMONT': 162500.0, 'AHIN': 97925.0, 'AISCHE-EN-REFAIL': 338566.6666666667, 'AISEAU': 4000.0, 'AISEAU-PRESLES': 308166.6666666667, 'AISEMONT': 

On trie le dictionnaire de la plus petite à la plus grande moyenne.

In [253]:
sorted_locality_dict = dict(sorted(locality_dict.items(), key=lambda item: item[1]))
#print(sorted_locality_dict)

On remplace la moyenne par un int pour pouvoir mapper après.

In [254]:
value_mapping = 1
for locality in sorted_locality_dict:
    sorted_locality_dict[locality] = value_mapping
    value_mapping += 1
#print(sorted_locality_dict)

In [255]:
df['Locality_num'] = df['Locality'].map(sorted_locality_dict)
df = df.dropna(subset=['Locality_num']) 

C:\Users\NGIRA\AppData\Local\Temp\ipykernel_20408\2233098269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Locality_num'] = df['Locality'].map(sorted_locality_dict)


## Column removing

We'll remove columns that provide a redundant information or are categorical and save the cleaned dataset.

In [256]:
clean_data = df.drop(columns=['Url','Country','BathroomCount','Garden','Locality','Kitchen','PEB','PostalCode','PropertyId','RoomCount','ShowerCount','StateOfBuilding','SubtypeOfProperty','TypeOfSale','Province','District','ConstructionYear','Fireplace','MonthlyCharges','ToiletCount','Terrace','Furnished','SwimmingPool','FloodingZone','Region','NumberOfFacades'])
clean_data.to_csv("clean_data.csv")